In [2]:
#Basic imports
import os
import matplotlib.pyplot as plt
import PIL
import essentia.standard as ess
import essentia
import numpy as np
import IPython.display as ipd
from random import randint

In [5]:
count = 0
count_songs = 0
labels = []
labels_dict = dict()


# iterate over directories in '../../'
# each directory is a label
# store the in labels variable
for root, dirs, files in os.walk ('../gnaimeh/boc_60s/'):
    count += 1
    if count==1:
        labels = [label for label in dirs if not label.startswith('.')]
    else:
        break

for l in labels:
    # count the number of songs in labels directories
    # file is considered to be song if it ends with .mp3 and doesn't
    # start with .
    count_songs = 0
    for root, dirs, files in os.walk('../gnaimeh/boc_60s/'+l):
        for file in files:
            if file.endswith('c3.mp3'):
                count_songs += 1

    # store label and it's count in labels_dict["name"] and
    # labels_dict["number_songs"] lists
    print(l, " songs ", count_songs)
    if "name" in labels_dict:
        labels_dict["name"].append(l)
        labels_dict['number_songs'].append(count_songs)
    else:
        labels_dict["name"] = [l]
        labels_dict["number_songs"] = [count_songs]

suaraarchive  songs  568
mordarchive  songs  384
defectedarchive  songs  1485
stilvortalentarchive  songs  561
bar25archive  songs  412
littlehelpersarchive  songs  350
kompaktarchive  songs  298


In [6]:
img_dir_name = '../gnaimeh/unbalanced_1mn_only/'

if not os.path.exists(img_dir_name):
    os.mkdir(img_dir_name)
    print("Directory ", img_dir_name, " Created ")
else:
    print("Directory ", img_dir_name, " already exists")

Directory  ../gnaimeh/unbalanced_1mn_only/  Created 


In [28]:
minute_1_folder = '../gnaimeh/unbalanced_1mn_only/'
if not os.path.exists(minute_1_folder):
    os.mkdir(minute_1_folder)
    print("Directory ", minute_1_folder, " Created ")
else:
    print("Directory ", minute_1_folder, " already exists")
 

minute_1_folder_train = '../gnaimeh/unbalanced_1mn_only/train'
if not os.path.exists(minute_1_folder_train):
    os.mkdir(minute_1_folder_train)
    print("Directory ", minute_1_folder_train, " Created ")
else:
    print("Directory ", minute_1_folder_train, " already exists")
    
    
minute_1_folder_test = '../gnaimeh/unbalanced_1mn_only/test'
if not os.path.exists(minute_1_folder_test):
    os.mkdir(minute_1_folder_test)
    print("Directory ", minute_1_folder_test, " Created ")
else:
    print("Directory ", minute_1_folder_test, " already exists")

Directory  ../gnaimeh/unbalanced_1mn_only/  already exists
Directory  ../gnaimeh/unbalanced_1mn_only/train  already exists
Directory  ../gnaimeh/unbalanced_1mn_only/test  already exists


In [22]:
split = 'train'

data_file = '../gnaimeh/unbalanced_1mn_only/'+split+'_complete.csv'
with open(data_file, 'w') as writer:
    line2write = 'path,label\n'
    writer.write(line2write)
    for l in labels:
        print(l)
        count = 0
        for root, dirs, files in os.walk ('../gnaimeh/unbalanced_1mn_only/'+split+'/'+l):
            for file in files:
                line2write = root+'/'+file+','+l+'\n'
                writer.write(line2write)
            

suaraarchive
mordarchive
defectedarchive
stilvortalentarchive
bar25archive
littlehelpersarchive
kompaktarchive


In [23]:
split = 'test'

data_file = '../gnaimeh/unbalanced_1mn_only/'+split+'_complete.csv'
with open(data_file, 'w') as writer:
    line2write = 'path,label\n'
    writer.write(line2write)
    for l in labels:
        print(l)
        count = 0
        for root, dirs, files in os.walk ('../gnaimeh/unbalanced_1mn_only//'+split+'/'+l):
            for file in files:
                line2write = root+'/'+file+','+l+'\n'
                writer.write(line2write)

suaraarchive
mordarchive
defectedarchive
stilvortalentarchive
bar25archive
littlehelpersarchive
kompaktarchive


In [31]:
countlabel = 0
count = 0
nfiles = min(labels_dict["number_songs"])
fs = 44100
sample_1m = fs*60   
    
# Start iteration through labels folders
for l in labels_dict['name']:
    if not os.path.exists('../gnaimeh/unbalanced_1mn_only_train'+'/'+l):
        os.mkdir(../gnaimeh/unbalanced_1mn_only_train+'/'+l)
        os.mkdir('../unbalanced_1mn_only_test'+'/'+l)
        print("Directory ", '../gnaimeh/unbalanced_1mn_only_train'+'/'+l, " Created ")
        print("Directory ", '../unbalanced_1mn_only_test'+'/'+l, " Created ")
    else:
        print("Directory ", '../unbalanced_1mn_only_test'+'/'+l, " already exists")
        print("Directory ", '../gnaimeh/unbalanced_1mn_only_train'+'/'+l, " already exists")
        print('ANALIZING NEW LABEL... ',l)
    count = 0    
    count_chunk = 0
    for root, dirs, files in os.walk (main_dir+l):
        # Check if counter of files per label and reset
        # Iterate over files in label l
        for file in files: 
            if not file.startswith('.') and file.endswith('c3.mp3'):                                  
                if count < nfiles:
                    count += 1
                    # Reading audio file 
                    x = ess.MonoLoader(filename=root+'/'+file, sampleRate=fs)()

                    # Append audio file choppend in a list
                    X = []
                    for i in range(x.size//sample_1m):
                        X.append(x[i*sample_1m:i*sample_1m+sample_1m])

                    # Initilization of spectrum, window and melbands
                    spectrum = ess.Spectrum()
                    windowing = ess.Windowing(type='blackmanharris62', zeroPadding=2048)
                    melbands = ess.MelBands(numberBands=96, lowFrequencyBound=0, highFrequencyBound=11000)

                    pool = essentia.Pool()
                    amp2db = ess.UnaryOperator(type='lin2db', scale=2)

                    # Mel band calculation for each chunk
                    for x in X:
                        count_chunk+=1;
                        pool = essentia.Pool()
                        for frame in ess.FrameGenerator(x, frameSize=2048, hopSize=1024):
                            frame_spectrum = spectrum(windowing(frame))
                            frame_mel = melbands(frame_spectrum)

                            pool.add('mel96_db', amp2db(frame_mel))  

                        # Save as training set 80% of each of the labels and as test set the 20%
                        if count <= 244:
                            print('Train: ',count,' | Chunk: ', count_chunk)
                            np.save('../unbalanced_1mn_only_train' + '/' + l + '/' + str(count_chunk), pool['mel96_db'].T)
                        else:
                            np.save('../unbalanced_1mn_only_test' + '/' + l + '/' + str(count_chunk), pool['mel96_db'].T)
                            print('Test: ',count,' | Chunk: ', count_chunk)


                    if count % 20 == 0:
                        print(count, " files processed, current file: ",file)
                        print(count_chunk, " chunks processed, current file: ",file)
                    



FileNotFoundError: [Errno 2] No such file or directory: '../gnaimeh/unbalanced_1mn_only_train/suaraarchive'